![header](../../img/logo.svg)

**LSTM Machine Learning Model Demo**

In [ ]:
# System libraries
import os
from datetime import timedelta

# Data processing and visulization libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Neural Network library
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Technials Analysis library
os.chdir("../technicals")
import technicals

# Market data connection library
os.chdir("../marketdata")
import alpaca

---

## Model Training

### *Market Data*

In [ ]:
# Set market data date range 
from datetime import date, datetime, timedelta

end_date  = datetime.now()
start_date  = (end_date - timedelta(days=1000))

start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

print(f"Start date : {start_date}")
print(f"End date : {end_date}")

In [ ]:
# Load the dataset
test_tickers = ["KO"]

In [ ]:
ohlcv_df = alpaca.ohlcv(test_tickers, start_date=start_date, end_date=end_date)
tech_ind = technicals.TechnicalAnalysis(ohlcv_df)

df = tech_ind.get_all_technicals(test_tickers[0])
df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
df.head()

---

### *Data pre-processing : Scaling*

#### Scaling using [RobustScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html):

Scale features using statistics that are robust to outliers.

This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. Median and interquartile range are then stored to be used on later data using the transform method.

Standardization of a dataset is a common requirement for many machine learning estimators. Typically this is done by removing the mean and scaling to unit variance. However, outliers can often influence the sample mean / variance in a negative way. In such cases, the median and the interquartile range often give better results.

In [ ]:
# scale fitting the close prices separately for inverse_transformations purposes later
close_scaler = RobustScaler()
close_scaler.fit(df[['close']])

In [ ]:
scaler = RobustScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

In [ ]:
df.head()

---

### *LSTM Model Helper Functions*

1. split_sequence() - splits multivariate time sequences for LSTM training
2. add_hidded_layers() - adds requested number of LSTM hidden layers with requested number of nodes
3. validater() - validates training results
4. rmse() - returns root-mean-squared error

In [ ]:
# LSTM Helper Functions
def split_sequence(sequence, n_in, n_out):
    '''
    Splits the multivariate time sequence

    Parameters
    ----------
    sequence : np.array
        numpy array of the dataframe used to train the model

    Returns
    -------
    X, y : np.array
        Time sequence values for X and y portions of the dataset
    '''
    
    # creating a list for both variables
    X, y = [], []
    
    for i in range(len(sequence)):
        
        # finding the end of the current sequence
        end = i + n_in
        out_end = end + n_out
        
        # breaking out of the loop if we have exceeded the dataset's length
        if out_end > len(sequence):
            break
        
        # splitting the sequences into: x = past prices and indicators, y = prices ahead
        sequence_x, sequence_y = sequence[i:end, :], sequence[end:out_end, 0]
        
        X.append(sequence_x)
        y.append(sequence_y)
    
    return np.array(X), np.array(y)
    
    
def add_hidden_layers(n_layers, n_nodes, activation, drop=None, drop_rate=.5):
    '''
    Creates a specific amount of hidden layers for the model

    Parameters
    ----------
    n_layers : int
        number of layers to be added to the model

    n_nodes : int
        number of nodes to be added to each layer

    activation : str
        activation function used by each layers in the model
        Full list of all activation functions: 
            https://www.tensorflow.org/api_docs/python/tf/keras/activations

    drop : int
        every n-th hidden layer after which a Dropout layer to be added

    drop_rate : float
        rate for each Dropout layer
        default = 0.5

    '''
    
    # creating the specified number of hidden layers with the specified number of nodes
    for x in range(1,n_layers+1):
        model.add(LSTM(n_nodes, activation=activation, return_sequences=True))

        # adds a Dropout layer after every n-th hidden layer
        try:
            if x % drop == 0:
                model.add(Dropout(drop_rate))
        except:
            pass
    
    
def validater():
    '''
    Creates predicted values.

    Returns
    -------
    predictions : pd.DataFrame
        Predicted values for the model
    '''
    
    # Creating an empty DF to store the predictions
    predictions = pd.DataFrame(index=df.index, columns=[df.columns[0]])

    for i in range(n_in, len(df)-n_in, n_out):
        # Creating rolling intervals to predict off of
        x = df[-i - n_in:-i]

        # Predicting using rolling intervals
        y_pred = model.predict(np.array(x).reshape(1, n_in, n_features))

        # Transforming values back to their normal prices
        y_pred = close_scaler.inverse_transform(y_pred)[0]

        # DF to store the values and append later, frequency uses business days
        pred_df = pd.DataFrame(y_pred, 
                               index=pd.date_range(start=x.index[-1], 
                                                   periods=len(y_pred), 
                                                   freq="B"),
                               columns=[x.columns[0]])

        # Updating the predictions DF
        predictions.update(pred_df)
        
    return predictions


def rmse(df_actual, df_predicted):
    '''
    Calculates the RMS (root mean square) error between the two pd.Dataframes
    
    Parameters
    ----------
    df_actual : pd.DataFrame
        DataFrame with Actual Closing prices
    df_predicted : pd.DataFrame
        DataFrame with Predicted Closing prices
    
    '''
    df = df_actual.copy()
    df['close_pred'] = df_predicted['close']
    df.dropna(inplace=True)
    df['diff'] = df['close']- df['close_pred']
    rms = (df[['diff']]**2).mean()
    error = float(np.sqrt(rms))
    
    return error

### *LSTM Visualization Functions*

1. visualize_training_results() - plots validation loss, loss, validation accuracy, and accuracy
2. visualize_training_price() - plots actual and predicted Closing stock price

In [ ]:
def visualize_training_results(results, model_loss_filename : str, model_accuracy_filename : str):
    """
    Plots the loss and accuracy for the training and testing data
    """
    history = results.history
    plt.figure(figsize=(16,5))
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.savefig(model_loss_filename)
    plt.show()
    
    plt.figure(figsize=(16,5))
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.savefig(model_accuracy_filename)
    plt.show()
    
def visualize_training_price(filename):
    '''
    Visualizes Actual vs. Predicted stock price
    '''

    # Transforming the actual values to their original price
    actual = pd.DataFrame(close_scaler.inverse_transform(df[["close"]]), 
                          index=df.index, 
                          columns=[df.columns[0]])

    # Getting a DF of the predicted values to validate against
    predictions = validater()

    # Printing the RMSE
    print(f"RMSE: {rmse(actual, predictions)}")

    # Plotting
    plt.figure(figsize=(16,6))

    # Plotting those predictions
    plt.plot(predictions, label='Predicted')

    # Plotting the actual values
    plt.plot(actual, label='Actual')

    plt.title(f"Predicted vs Actual Closing Prices")
    plt.ylabel("Price")
    plt.legend()
    plt.savefig(filename)
    plt.show()

---

### *Building LSTM Model*

Builds an LSMT model, compiles and fits it.

* Looks back 100 days to predict 14 days
* Sequential() model - groups a linear stack of layers into a [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model) : [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
* Uses Hyperbolic tangent activation function : [tf.keras.activations.tanh](https://www.tensorflow.org/api_docs/python/tf/keras/activations/tanh)
* Composed of LSTM input and hidden layers - Long Short-Term Memory layer - [Hochreiter 1997](https://dl.acm.org/doi/10.1162/neco.1997.9.8.1735): [tf.keras.layers.LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
* Uses Adam optimzer - a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments : [tf.keras.optimizers.Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)
* Uses Mean Squared Error loss function - computes the mean of squares of errors between labels and predictions: [tf.keras.losses.MeanSquaredError](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
* Model is trained for 50 epochs using 128 unit batch size

In [ ]:
# n periods looking back to learn
n_in  = 100

# n periods to predict
n_out = 14

# n features 
n_features = df.shape[1]

# X, y split
X, y = split_sequence(df.to_numpy(), n_in, n_out)

# Dataframe for predictions
predictions = None

In [ ]:
# instatiating the model
model = Sequential()

In [ ]:
# activation
activation_func = "tanh"

In [ ]:
# add input layer
model.add(LSTM(90, 
               activation=activation_func, 
               return_sequences=True, 
               input_shape=(n_in, n_features)))

In [ ]:
# add hidden layers
add_hidden_layers(n_layers=1, 
                  n_nodes=30, 
                  activation=activation_func)

In [ ]:
# add final Hidden layer
model.add(LSTM(60, activation=activation_func))

In [ ]:
# add output layer
model.add(Dense(n_out))

In [ ]:
# print model summary
model.summary()

In [ ]:
# compile model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# fit and train the model
res = model.fit(X, y, epochs=50, batch_size=128, validation_split=0.1)

---

### *Visualize Training Results*

In [ ]:
visualize_training_results(res, model_loss_filename="model_loss_KO.png", model_accuracy_filename="model_accuracy_KO.png")

In [ ]:
visualize_training_price(filename="pred_prices_KO.png")

---

### *Save model*

In [ ]:
os.chdir("../ml/savedmodels")

In [ ]:
model.save("proj2_demo_model_KO.h5")

---

## Forecasting future stock price

### *Get market data*

In [ ]:
# Set market data date range 
from datetime import date, datetime, timedelta

pred_end_date  = datetime.now()
pred_start_date  = (pred_end_date - timedelta(days=200))

pred_start_date = pred_start_date.strftime('%Y-%m-%d')
pred_end_date = pred_end_date.strftime('%Y-%m-%d')

print(f"Forecast start date : {pred_start_date}")
print(f"Forecast end date : {pred_end_date}")

In [ ]:
# Load the dataset
pred_ohlcv_df = alpaca.ohlcv(test_tickers, start_date=pred_start_date, end_date=pred_end_date)

pred_tech_ind = technicals.TechnicalAnalysis(pred_ohlcv_df)

pred_df = pred_tech_ind.get_all_technicals(test_tickers[0])

pred_df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

pred_df.head()

### *Data pre-processing : Scaling*

In [ ]:
pred_scaler = RobustScaler()
transformed_pred_df = pd.DataFrame(pred_scaler.fit_transform(pred_df), 
                       columns=pred_df.columns, 
                       index=pred_df.index).tail(n_in)

In [ ]:
transformed_pred_df.head()

In [ ]:
# transform technical analysis data to np.array
pred_arr = np.array(transformed_pred_df).reshape(1, n_in, n_features)

### *Forecast stock price*

In [ ]:
# load model
from tensorflow.keras.models import load_model
forecast_model = load_model("proj2_demo_model_KO.h5")

In [ ]:
# predicting off of the new data
pred_y = forecast_model.predict(pred_arr)

In [ ]:
# inverse_transform the predicted values back to original scale
pred_y = close_scaler.inverse_transform(pred_y)[0]

In [ ]:
# parse perdicted values to pd.DataFrame, adjust date scale (index)
preds = pd.DataFrame(pred_y, 
                     index=pd.date_range(start=df.index[-1]+timedelta(days=1), 
                                         periods=len(pred_y), 
                                         freq="B"), 
                     columns=[df.columns[0]])

### *Plot the results*

In [ ]:
# get actual historical prices
actual = pred_df[["close"]].tail(n_in)

In [ ]:
# get company name from ticker
# Source: https://stackoverflow.com/questions/38967533/retrieve-company-name-with-ticker-symbol-input-yahoo-or-google-api
import requests

def get_company_name(ticker):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(ticker)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == ticker:
            return x['name']

In [ ]:
# plot results
plt.figure(figsize=(16,6))
plt.plot(actual, label="Actual Prices")
plt.plot(preds, label="Predicted Prices")
plt.ylabel("Price")
plt.xlabel("Dates")
plt.title(f"Forecasting next {len(pred_y)} days for {get_company_name(test_tickers[0])}")
plt.legend()
plt.savefig("model_forecast_KO.png")
plt.show()